In [1]:
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns 
%matplotlib inline
%config InlineBackend.figure_format='retina'
import cvxpy as cp
import time
import gurobipy
import pandas as pd

np.random.seed(230824)

(CVXPY) Jul 10 06:45:42 PM: Encountered unexpected exception importing solver PROXQP:
ImportError("dlopen(/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/cmeel.prefix/lib/python3.12/site-packages/proxsuite/instructionset.cpython-312-darwin.so, 0x0002): Library not loaded: @rpath/libc++.1.dylib\n  Referenced from: <1A57AE5C-C7A4-3364-A443-4BC8EA7A0D79> /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/cmeel.prefix/lib/python3.12/site-packages/proxsuite/instructionset.cpython-312-darwin.so\n  Reason: tried: '/Users/runner/miniconda3/envs/proxsuite/lib/libc++.1.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/runner/miniconda3/envs/proxsuite/lib/libc++.1.dylib' (no such file), '/var/folders/mj/dx2cc6hn6h9br7vt9hz4n06m0000gn/T/cmeel-0at3h7tz/whl/cmeel.prefix/lib/libc++.1.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/var/folders/mj/dx2cc6hn6h9br7vt9hz4n06m0000gn/T/cmeel-0at3h7tz/whl/cmeel.prefix/l

In [2]:
'''
data generation
'''

# dimensions
n,m,d = int(2e2),int(1e3),int(100)
TIME_LIMIT = 500
main_scale_beta = 25.0 
inflation_factor = .2 
rank_B_dev = min(4,d)


def data_generator(delta_beta=1,impact_B_dev = .2):
    
    # ref vectors
    bar_beta = np.random.randint(-int(max(1,abs(main_scale_beta))),int(max(1,abs(main_scale_beta)))+1,d)/int(max(1,abs(main_scale_beta)))
    norm_bar_beta = np.linalg.norm(bar_beta)
    pre_diag_B_ref = np.random.uniform(.25,1.0,d)
    diag_B_ref = pre_diag_B_ref / np.sum(pre_diag_B_ref) * d 
    mean_EV = np.mean(diag_B_ref)
    R_base = np.sqrt(np.sum(np.abs((diag_B_ref**(1/2))*np.ones(d))**2)*(1+inflation_factor))
    
    # deviations
    betas = np.outer(np.ones(n),bar_beta)
    for i in range(n):
        dev_i = np.random.normal(0,1,d)
        dev_i /= np.linalg.norm(dev_i)
        betas[i] += dev_i*delta_beta*norm_bar_beta
    
    B_devs = []
    for j in range(m):
        # Generate a random matrix with standard normal entries
        H = np.random.standard_normal(size=(d, d))
        # Compute QR decomposition
        Q, R = np.linalg.qr(H)
        # Adjust signs of Q to ensure uniform distribution
        diag_signs = np.sign(np.diag(R))
        diag_signs[diag_signs == 0] = 1  # Handle zero signs (unlikely)
        Q = Q * diag_signs  # Multiply each column by its sign
        B_devs.append(Q[:rank_B_dev,:])


    return diag_B_ref,betas,B_devs

In [3]:
dic_results = {'method':[],'rel_dev_obj':[],'rel_dev_cstr':[],'time':[],'F_hat':[],'rep':[],'F_check':[]}
N_reps = int(10)

my_env = gurobipy.Env()
my_env.setParam('TimeLimit', TIME_LIMIT) # in seconds


for db in [.1,.5,1,5,10]:
    for iBd in [.1,.5,2]:

        print('=> data generation for db = '+str(db)+' and iBd = '+str(iBd))
        print(' ')
        
        diag_B_ref,betas,B_devs = data_generator(delta_beta=db,impact_B_dev =iBd)
        mean_EV = np.mean(diag_B_ref)
        R_base = np.sqrt(np.sum(np.abs(diag_B_ref*np.ones(d))**2)*(1+inflation_factor))

        '''
        cstr generator
        ''' 
        def DOMGEN(cvxpy_var,S=np.arange(m),sc_factor=iBd):
            buf = []
            for j in S:
                buf.append(cp.sum_squares(cp.multiply(diag_B_ref**(1/2),cvxpy_var))+sc_factor*mean_EV*cp.sum_squares(B_devs[int(j)]@cvxpy_var)-R_base**2<=0)
            return buf.copy()
        '''
        helper numpy functions
        '''
        def c_values(x,S=np.arange(m),sc_factor=iBd):
            inners = np.array(B_devs)[S]@x
            return np.sum((diag_B_ref**(1/2)*x)**2)-R_base**2 + sc_factor*mean_EV*np.sum(inners**2,1)
        
        def active_cstr(x,sc_factor_bis=iBd,feasactive=1e-4):
            cvals = c_values(x,sc_factor=sc_factor_bis)
            return np.where(cvals>=-feasactive)[0]
        
        def f_values(x,H=np.arange(n)):
            return betas[H]@x
        
        def active_f(x,rho=1e-3):
            fvals = f_values(x)
            return np.where(fvals<=np.min(fvals)+rho)[0]
        
        def F(x,feastol=1e-5):
            if np.max(c_values(x))>=feastol:
                return np.inf
            else:
                return np.min(f_values(x))


        ############################ solving time

        ## CVXPY global 
        x_glob = cp.Variable(d)
        cstrs_full = DOMGEN(x_glob)
        lin_param_glob = cp.Parameter(d)
        eta_glob = cp.Variable(1)
        t_glob = cp.Variable(n,boolean=True)
        
        bigMs = np.zeros((n,n))
        
        for i_plus in range(n):
            for i in range(n):
                # closed-forms here
                if i_plus!=i:
                    delta_betas = betas[i_plus]-betas[i]
                    tilde_lin_c = delta_betas*(diag_B_ref**(-1/2))
                    bigMs[i_plus,i] = np.linalg.norm(tilde_lin_c)*R_base
        
        cstrs_full += [cp.sum(t_glob)==1]
        
        for i_plus in range(n):
            cstrs_full += [betas[i_plus]@x_glob<=eta_glob[0]+bigMs[i_plus]@t_glob]
        
        prob_glob = cp.Problem(cp.Minimize(eta_glob[0]),cstrs_full)
        t_start = time.time()
        prob_glob.solve(solver=cp.GUROBI,verbose=True,env=my_env)
        t_end = time.time()

        dic_results['method'].append('bigM (direct)')
        dic_results['rel_dev_obj'].append(db)
        dic_results['rel_dev_cstr'].append(iBd)
        dic_results['time'].append(t_end-t_start)
        dic_results['F_hat'].append(prob_glob.value)
        dic_results['rep'].append(1)
        if prob_glob.status==cp.OPTIMAL:
            dic_results['F_check'].append(prob_glob.value)
        else:
            grb_model = prob_glob.solver_stats.extra_stats
            best_bound = grb_model.ObjBound
            dic_results['F_check'].append(best_bound)

        print(' ')
        print('end of glob')
        print(' ')


        ## CVXPY ULO 
        x_ulo = cp.Variable(d)
        cstrs_full = DOMGEN(x_ulo)
        lin_param_ulo = cp.Parameter(d)
        obj_ulo = cp.Minimize(lin_param_ulo@x_ulo)
        prob_ulo_full = cp.Problem(obj_ulo,cstrs_full)
        
        '''
        inplace version of ULO
        '''
        def ULO(i_hat,epsilon_tilde=1e-2,epsilon=1e-2,verb_lvl=2,time_lim=TIME_LIMIT):
        
            if verb_lvl>0:
                print('ULO start')
                print(' ')
            
            ## init.
            H,S = [],[]
            F_check,F_hat = -np.inf,np.inf
            rho = 1e-3
            i_star = None
            bar_x = None
            x_out = None
            k = 0
        
            nus = np.inf*np.ones(n)

            inner_time_start = time.time()
        
            ## oracles set-up 
            
            ## main loop
            while k==0 or F_hat-F_check>max(epsilon,max(1,abs(F_hat))*epsilon_tilde):
        
                # -> phase (a) <- 
                R,V = np.inf,[i_hat]
                while len(V)>0:
                    bar_i = np.random.choice(V)
                    lin_param_ulo.value = betas[bar_i]
                    H.append(bar_i)
                    ### start oracle call 
                    if verb_lvl>=1:
                        print('tested piece #'+str(bar_i))
                    prob_ulo_full.solve(solver=cp.MOSEK,warm_start=True)
                    x_tilde = x_ulo.value
                    nu_bar_i = prob_ulo_full.value
                    if verb_lvl>=1:
                        print('value: '+str(nu_bar_i))
                    nus[bar_i] = nu_bar_i
                    ### end oracle call 
                    if nu_bar_i >= R:
                        V = np.setdiff1d(V,[bar_i])
                    else:
                        V = np.setdiff1d(active_f(x_tilde,rho),H)
                        R = F(x_tilde)
                        i_star = bar_i
                        bar_x = x_tilde

                    if time.time()-inner_time_start>time_lim:
                        if verb_lvl>0:
                            print('time limit reached...')
                        return x_out,F_hat,F_check
        
                ### update of the set of selected constraints
                S = np.union1d(S,[np.random.choice(np.setdiff1d(np.arange(m),S))])
                S = np.union1d(S,active_cstr(bar_x))
        
                val_bar_x = F(bar_x)
                if val_bar_x<F_hat:
                    F_hat,x_out = val_bar_x,bar_x
        
                if F_hat-F_check<=max(epsilon,max(1,abs(F_hat))*epsilon_tilde):
                    if verb_lvl>0:
                        print(' ')
                        print('early convergence within iteration #'+str(k))
                    return x_out,F_hat,F_check
        
                if verb_lvl>=1:
                    print('F_hat = '+str(F_hat))
                    if verb_lvl>=2:
                        print('|S| = '+str(len(S)))
        
                # -> phase (b) <-
                for i in np.setdiff1d(range(n),H):
                    ### start oracle call 
                    cstrs_S = DOMGEN(x_ulo,S)
                    lin_param_ulo.value = betas[i]
                    prob_ulo_S = cp.Problem(obj_ulo,cstrs_S)
                    if verb_lvl>=3:
                        print('tested (LB) piece #'+str(i))
                    prob_ulo_S.solve(solver=cp.MOSEK,warm_start=True)
                    nus[i] = prob_ulo_S.value
                    if verb_lvl>=3:
                        print('value: '+str(nus[i]))
                    ### end oracle call 
                    if time.time()-inner_time_start>time_lim:
                        if verb_lvl>0:
                            print('time limit reached...')
                        return x_out,F_hat,F_check
                    
                i_hat = np.argmin(nus)
                F_check = max(F_check,nus[i_hat])
                if verb_lvl>=1:
                    print('F_check = '+str(F_check))
        
                k += 1
                
            if verb_lvl>0:
                print(' ')
                print('convergence after '+str(k)+' iterations')
                
            return x_out,F_hat,F_check

        for _ in range(N_reps):
            t_start = time.time()
            x_out,F_hat,F_check = ULO(i_hat=np.random.choice(np.arange(n)),epsilon_tilde=1e-6,epsilon=1e-2)
            t_end = time.time()
            dic_results['method'].append('ULO')
            dic_results['rel_dev_obj'].append(db)
            dic_results['rel_dev_cstr'].append(iBd)
            dic_results['time'].append(t_end-t_start)
            dic_results['F_hat'].append(F_hat)
            dic_results['F_check'].append(F_check)
            dic_results['rep'].append(_)

        print(' ')
        print('end of ULO')
        print(' ')
    
        ## CVXPY ES
        x_es = cp.Variable(d)
        cstrs_full = DOMGEN(x_es)
        lin_param_es = cp.Parameter(d)
        obj_es = cp.Minimize(lin_param_es@x_es)
        prob_es_full = cp.Problem(obj_es,cstrs_full)
        times = []
        for i in np.random.choice(np.arange(n),size=int(n/10),replace=False):
            lin_param_es.value = betas[i]
            t_start = time.time()
            prob_es_full.solve(solver=cp.GUROBI)
            t_end = time.time()
            print('trial #i = '+str(i)+' took '+str(t_end-t_start)+' [s]')
            times.append(t_end-t_start)
        dic_results['method'].append('ES')
        dic_results['rel_dev_obj'].append(db)
        dic_results['rel_dev_cstr'].append(iBd)
        dic_results['time'].append(np.mean(times)*n)
        dic_results['F_hat'].append(F_hat)
        dic_results['F_check'].append(F_hat) # we assume F_hat is the good one ;) 
        dic_results['rep'].append(1)

        print(' ')
        print('end of ES')
        print(' ')

Set parameter Username
Academic license - for non-commercial use only - expires 2026-07-09
Set parameter TimeLimit to value 500
=> data generation for db = 0.1 and iBd = 0.1
 
                                     CVXPY                                     
                                     v1.6.5                                    
(CVXPY) Jul 10 06:45:45 PM: Your problem has 301 variables, 1201 constraints, and 0 parameters.
(CVXPY) Jul 10 06:45:45 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 10 06:45:45 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 10 06:45:45 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jul 10 06:45:45 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation       

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


 
end of glob
 
ULO start
 
tested piece #137
value: -63.93275663037058
tested piece #12
value: -65.19393331549723
F_hat = -65.19393331549723
|S| = 3
F_check = -65.19393331549723
 
convergence after 1 iterations
ULO start
 
tested piece #162
value: -62.9775688981044
tested piece #12
value: -65.19393331549723
F_hat = -65.19393331549723
|S| = 3
F_check = -65.19393331549723
 
convergence after 1 iterations
ULO start
 
tested piece #175
value: -62.48210444058179
tested piece #12
value: -65.19393331549723
F_hat = -65.19393331549723
|S| = 3
F_check = -65.19393331549723
 
convergence after 1 iterations
ULO start
 
tested piece #93
value: -63.55789217515065
tested piece #12
value: -65.19393331549723
F_hat = -65.19393331549723
|S| = 3
F_check = -65.19393331549723
 
convergence after 1 iterations
ULO start
 
tested piece #104
value: -62.572507440179926
tested piece #12
value: -65.19393331549723
F_hat = -65.19393331549723
|S| = 3
F_check = -65.19393331549723
 
convergence after 1 iterations
ULO s


KeyboardInterrupt



In [19]:
my_df_results = pd.DataFrame(data=dic_results)

In [21]:
my_df_results.to_csv('ulo_extended_results.csv')

In [22]:
my_df_results

,method,rel_dev_obj,rel_dev_cstr,time,F_hat,rep,F_check
0,bigM (direct),0.1,0.1,519.785641,-64.819773,1,-70.478789
1,ULO,0.1,0.1,25.155650,-65.193933,0,-65.193933
2,ULO,0.1,0.1,8.085698,-65.193933,1,-65.193933
3,ULO,0.1,0.1,7.821373,-65.193933,2,-65.193933
4,ULO,0.1,0.1,7.599181,-65.193933,3,-65.193933
...,...,...,...,...,...,...,...
166,ULO,10.0,0.5,256.229927,-722.658738,9,-722.658738
167,ES,10.0,0.5,2263.437004,-722.658738,1,-722.658738
168,bigM (direct),10.0,2.0,518.513920,-688.790191,1,-1089.689768
169,ULO,10.0,2.0,2735.594461,-688.789841,0,-688.789841


In [45]:
for value in my_df_results['rel_dev_obj'].unique():
    for subvalue in my_df_results['rel_dev_cstr'].unique():
        print('set-up : '+str([value,subvalue]))
        sub_df = my_df_results[(my_df_results['rel_dev_obj']==value) & (my_df_results['rel_dev_cstr']==subvalue)]
        vec = np.array((sub_df['F_hat']-sub_df['F_check'])/np.maximum(1,np.abs(sub_df['F_hat'])))
        vec = np.array([vec_elem if np.abs(vec_elem)>1e-7 else 0 for vec_elem in vec])    
        print('rel-gap for glob_opt: '+str(vec[0]*100)[:5]+' % ')
        for met in sub_df['method'].unique():
            print('mean time for '+str(met)+' : '+str(np.mean(sub_df[sub_df['method']==met]['time'])))
        print(' ')

set-up : [0.1, 0.1]
rel-gap for glob_opt: 8.730 % 
mean time for bigM (direct) : 519.7856409549713
mean time for ULO : 9.527860355377197
mean time for ES : 2636.2521028518677
 
set-up : [0.1, 0.5]
rel-gap for glob_opt: 9.049 % 
mean time for bigM (direct) : 520.355801820755
mean time for ULO : 9.946706581115723
mean time for ES : 2738.6419320106506
 
set-up : [0.1, 2.0]
rel-gap for glob_opt: 10.31 % 
mean time for bigM (direct) : 520.0623910427094
mean time for ULO : 31.91093544960022
mean time for ES : 2723.765239715576
 
set-up : [0.5, 0.1]
rel-gap for glob_opt: 36.35 % 
mean time for bigM (direct) : 521.5730330944061
mean time for ULO : 31.137296867370605
mean time for ES : 2762.143738269806
 
set-up : [0.5, 0.5]
rel-gap for glob_opt: 33.40 % 
mean time for bigM (direct) : 521.8475389480591
mean time for ULO : 55.63224003314972
mean time for ES : 2350.852963924408
 
set-up : [0.5, 2.0]
rel-gap for glob_opt: 42.96 % 
mean time for bigM (direct) : 516.8693680763245
mean time for ULO :